In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(123)
if device == 'cuda':
    torch.cuda.manual_seed_all(123)

learning_rate = 0.1
training_epochs = 30
batch_size = 100

mnist_train = dsets.MNIST(root='./data',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

train_set, val_set = torch.utils.data.random_split(mnist_train, [54000, 6000])

test_set = dsets.MNIST(root='./data',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)


val_dataloader = torch.utils.data.DataLoader(dataset=val_set, batch_size=6000)
test_X= test_set.test_data.view(-1, 28 * 28).float().to(device)
test_Y=test_set.test_labels.to(device)
val_X, val_Y = next(iter(val_dataloader))
val_X=val_X.view(-1, 28 * 28).float().to(device)
val_Y=val_Y.to(device)

data_loader = torch.utils.data.DataLoader(dataset=train_set,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [ ]:

linear1 = torch.nn.Linear(784, 1024, bias=True)
linear2 = torch.nn.Linear(1024, 10, bias=True)
relu = torch.nn.ReLU()

torch.nn.init.kaiming_uniform_(linear1.weight)
torch.nn.init.kaiming_uniform_(linear2.weight)

model = torch.nn.Sequential(linear1, relu, linear2).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
total_batch = len(data_loader)

testacc_list=[]
valacc_list=[]

for epoch in range(training_epochs):
    avg_cost = 0
  
    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    with torch.no_grad():

        prediction = model(val_X)
        correct_prediction = torch.argmax(prediction, 1) == val_Y
        val_accuracy = correct_prediction.float().mean()*100

        prediction = model(test_X)
        correct_prediction = torch.argmax(prediction, 1) == test_Y
        test_accuracy = correct_prediction.float().mean()*100

    testacc_list.append(test_accuracy)
    valacc_list.append(val_accuracy)
    print(f'Epoch:{epoch + 1:3d}, cost = {avg_cost:.9f}, test_acc:{test_accuracy}, val_acc:{val_accuracy}')


In [ ]:
epoch_line=[i for i in range(1,training_epochs+1)]

plt.plot(epoch_line, testacc_list,'r-',label='test_accuracy')
plt.plot(epoch_line, valacc_list,'b-', label='val_accuracy')
plt.xlabel(f'epoch(batch_size={batch_size})')
plt.ylabel('accuracy(%)')
plt.legend()
plt.show()